In [1]:
import numpy as np
from numpy import array
import math
import random

6. (5 баллов) Система координат камеры получается из мировой поворотом на 45 градусов вокруг оси z и трансляцией на 10 вдоль оси z. Внутренние параметры камеры: fx=fy=400, cx=960, cy=540. Написать программу, которая выводит на экран матрицу проекции камеры и целочисленные координаты пикселя на изображении, соответствующего трехмерной точке с координатами (10, -10, 100) в мировой системе координат.

In [2]:
fx = fy = 400     # дано
cx = 960
cy = 540
xyz = np.array([10, -10, 100, 1])
a6 = 45  

In [3]:
k6 = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
R_T = np.array([[math.cos(math.radians(a6)), -math.sin(math.radians(a6)), 0, 0], 
                [math.sin(math.radians(a6)), math.cos(math.radians(a6)), 0, 0], 
                [0, 0, 1, 10]]) #вращение вокруг оси z + трансляция по z

In [4]:
uvw = k6 @ R_T
print(f'Матрица проекции камеры:\n', uvw)

Матрица проекции камеры:
 [[ 2.82842712e+02 -2.82842712e+02  9.60000000e+02  9.60000000e+03]
 [ 2.82842712e+02  2.82842712e+02  5.40000000e+02  5.40000000e+03]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  1.00000000e+01]]


In [5]:
uvw0 = uvw @ xyz
uv = uvw0[:2] / uvw0[2]
print(f'Координаты пикселя:\n', np.around(uv).astype(int))

Координаты пикселя:
 [1011  540]


7. (5 баллов) Написать генератор пар двумерных точек для теста алгоритма, удовлетворяющих заданному заранее преобразованию гомографии H0. Используя прямое линейное преобразование (DLT) и SVD, написать программу, которая принимает на вход множество пар двумерных точек, и находит соответствующую этим парам матрицу гомографии H. Распечатать матрицы H0 и H.

In [6]:
def points_generator(num_points, H0):       # генератор точек
    x = np.random.rand(num_points, 2) 
    xp = np.hstack((x, np.ones((num_points, 1)))) @ (H0.T)
    xp = xp[:, :2] / xp[:, 2:]
    return x, xp


def DLT(x, x0): # прямое линейное преобразование (DLT)
    A = []
    # A = [
    #         x1  y1   1   0    0    0   -x1*xp1   -y1*xp1   -xp1;
    #         0    0   0   x1   y1   1   -x1*yp1   -y1*yp1   -yp1;
    #         x2  y2   1   0    0    0   -x2*xp2   -y2*xp2   -xp2;
    #         0    0   0   x2   y2   1   -x2*yp2   -y2*yp2   -yp2;
    #     ...       
    for i in range(len(x)):
        xi, yi = x[i, 0], x[i, 1]
        xp_i, yp_i = xp[i, 0], xp[i, 1]
        A.append([xi, yi, 1, 0, 0, 0, -xi * xp_i, -yi * xp_i, -xp_i])
        A.append([0, 0, 0, xi, yi, 1, -xi * yp_i, -yi * yp_i, -yp_i])
    A = np.array(A)
    u, s, vh = np.linalg.svd(A)     # SVD
    H = vh[-1, :].reshape(3, 3)
    return H

In [7]:
H0 = np.array([[1, 0, 100], [0, 0.5, 50], [0, 0, 1]])
x, xp = points_generator(10, H0)
H = DLT(x, xp)

print(f'Заданная матрица гомографии H0: \n', H0)
print(f'Матрица после преобразования H: \n', H)

Заданная матрица гомографии H0: 
 [[  1.    0.  100. ]
 [  0.    0.5  50. ]
 [  0.    0.    1. ]]
Матрица после преобразования H: 
 [[-8.94346703e-03 -2.09415515e-14 -8.94346703e-01]
 [ 2.27752052e-15 -4.47173352e-03 -4.47173352e-01]
 [-1.06237352e-16 -1.08470601e-16 -8.94346703e-03]]


8. (5 баллов) Модифицировать генератор из Задачи 7 так, чтобы он выдавал n=1000 соответствий, из которых 30% соответствуют заданному заранее преобразованию гомографии, а остальные взяты случайно (преобразованию гомографии не удовлетворяют). Реализовать нахождение преобразования гомографии H1, используя DLT и RANSAC. Распечатать H0 и H1.

In [8]:
H0 = np.array([[1, 0, 100], [0, 0.5, 50], [0, 0, 1]])

def get_proportion(n, p=0.3):   
    points1 = np.random.rand(n, 2)
    points2 = (points1 @ H0[:2, :2]) + H0[:2, 2]           # 30% соответствуют заданному заранее преобразованию гомографии
    points2[int(n * p):] += np.random.rand(int(n * (1 - p)), 2) # ост. преобразованию гомографии не удовлетворяют
    return points1, points2

def DLT(points1, points2):
    A = np.zeros((2 * points1.shape[0], 9))
    for i, (x1, y1) in enumerate(points1):
        A[2 * i] = [-x1, -y1, -1, 0, 0, 0, x1 * points2[i, 0], y1 * points2[i, 0], points2[i, 0]]
        A[2 * i + 1] = [0, 0, 0, -x1, -y1, -1, x1 * points2[i, 1], y1 * points2[i, 1], points2[i, 1]]
    U, S, V = np.linalg.svd(A)
    H = V[-1].reshape(3, 3)
    return H / H[-1, -1]

def RANSAC(points1, points2, iterations=100, threshold=0.1):
    max_inliers = 0
    best_H = None
    for i in range(iterations):
        indices = random.sample(range(points1.shape[0]), 4)
        sample_points1 = points1[indices]
        sample_points2 = points2[indices]
        H = DLT(sample_points1, sample_points2)
        inliers = np.sum(np.linalg.norm(np.dot(points1, H[:2, :2].T) + H[:2, 2].T - points2, axis=1) < threshold)
        if inliers > max_inliers:
            max_inliers = inliers
            best_H = H
    return best_H

In [9]:
points1, points2 = get_proportion(1000)  # n = 1000
H1 = RANSAC(points1, points2)

print(f'Матрица H0: \n', H0)
print(f'H1 (RANSAC): \n', H1)

Матрица H0: 
 [[  1.    0.  100. ]
 [  0.    0.5  50. ]
 [  0.    0.    1. ]]
H1 (RANSAC): 
 [[ 1.00000000e+00 -5.53453911e-11  1.00000000e+02]
 [ 3.36450735e-11  5.00000000e-01  5.00000000e+01]
 [ 6.63058838e-13 -5.39779283e-13  1.00000000e+00]]


9. (5 баллов) Два изображения одной и той же сцены сделаны одной и той же камерой с такой же матрицей внутренних параметров, как в Задаче 6. Второй снимок сделан после поворота камеры на 30 градусов вокруг оси x относительно начала координат системы отсчета, связанной с камерой (без сдвига). Найти матрицу гомографии между двумя изображениями.

In [10]:
fx = fy = 400     # дано
cx = 960
cy = 540
a = 30  
k = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
R = np.array([[1, 0, 0],
              [0, math.cos(math.radians(a)), -math.sin(math.radians(a))], 
              [0, math.sin(math.radians(a)), math.cos(math.radians(a))]]) # вращение вокруг оси x
H = k @ (R) @ (np.linalg.inv(k))
print(f'Матрица гомографии: \n', H)

Матрица гомографии: 
 [[ 1.00000000e+00  1.20000000e+00 -7.76615612e+02]
 [ 0.00000000e+00  1.54102540e+00 -5.64500000e+02]
 [ 0.00000000e+00  1.25000000e-03  1.91025404e-01]]


## Домашняя работа 3

13. (5 баллов) Первая камера повернута относительно второй на 45 градусов по оси z и сдвинута на 10 вдоль направления оси x второй камеры. У обеих камер матрицы внутренних параметров единичные. Найти и распечатать фундаментальную матрицу.

In [11]:
from scipy.spatial.transform import Rotation as R
anglez = np.pi/4
r1 = R.from_rotvec([0, 0, anglez]).as_matrix()
t1 = [10, 0, 0]
F1 = np.cross(t1, np.eye(3)).transpose() @ r1
print(f'Фундаментальная матрица: \n', F1)

Фундаментальная матрица: 
 [[  0.           0.           0.        ]
 [  0.           0.         -10.        ]
 [  7.07106781   7.07106781   0.        ]]


14. (5 баллов) Первая камера повернута относительно мировой системы координат на 45 градусов по оси z, а вторая – на -45 градусов по оси у и сдвинута на 10 в направлении оси x. У обеих камер матрицы внутренних параметров единичные. Найти и распечатать фундаментальную матрицу

In [12]:
a = 45
A_1 = np.array([[math.cos(math.radians(a)), -math.sin(math.radians(a)), 0], 
                [math.sin(math.radians(a)), math.cos(math.radians(a)), 0],     # вращение вокруг оси z
                [0, 0, 1]])

B_1 = np.array([[0], [0], [0]])

A_2 = np.array([[np.cos(math.radians(-a)), 0, np.sin(math.radians(-a))],
                [0, 1, 0],
                [-np.sin(math.radians(-a)), 0, np.cos(math.radians(-a))]])     # вращение вокруг оси y

B_2 = np.array([[10], [0], [0]]) # сдвиг по x

A = np.matmul(np.linalg.inv(A_2), A_1)
B = -1 * np.matmul(np.linalg.inv(A_2), B_2)

Fund_mx = np.cross(B, A, axisa=0, axisb=0, axisc=0)
print(f'Фундаментальная матрица: \n', Fund_mx)

Фундаментальная матрица: 
 [[-5. -5. -0.]
 [ 0.  0. 10.]
 [-5. -5.  0.]]


15. В задаче 14 найти и распечатать оба эпиполя

In [13]:
U_1 = np.array([10, 0, 0, 1]).T
U_2 = np.array([0, 0, 0, 1]).T
K = np.eye(3)

Q_1 = np.concatenate((np.linalg.inv(A_1), B_1), axis=1)
P_1 = K @ Q_1
Q_2 = np.concatenate((np.linalg.inv(A_2), B_2), axis=1)
P_2 = K @ Q_2

epipol1 = P_1 @ U_1
epipol2 = P_2 @ U_2

print(f'Эпиполи камер: ', epipol1, ', ', epipol2)

Эпиполи камер:  [ 7.07106781 -7.07106781  0.        ] ,  [10.  0.  0.]


16. В задаче 14 найти и распечатать эпиполярную линию, проходящую через точку (0,0) на первом изображении, и соответствующую ей линию на втором изображении.

In [14]:
point = np.array([0, 0, 1])
l2 = Fund_mx @ point
P_inv = np.linalg.pinv(P_1)
point2 = np.matmul(P_2, np.matmul(P_inv, point))
l = np.transpose(Fund_mx) @ point2

print(f'Эпиполярная линия: ', l)

Эпиполярная линия:  [-7.07106781 -7.07106781  0.        ]
